ニューラルネットワークは本来「網」なので、要素であるニューロン（神経）が文字通り網の目状に絡まっています。ただそういう構造は複雑で作る事が難しいため、構造（モデル）を簡略化します。その一つのモデルがシーケンシャル（Sequential）モデルです。シーケンシャルというのは「線形の」という意味で、線形というのは線のように一列につながっているという事です。その繋がる元は「レイヤー(Layer)」と呼ばれているニューロン（パーセプトロン（Perseptron））の塊です：
　レイヤーの中にあるパーセプトロンが次のレイヤーに信号を送る。これをシーケンシャルに続けていき、最後のレイヤーまで届ける。この割と単純なモデルがシーケンシャルモデルです。Sequentialモデルにはこのレイヤーをいくらでも直列に追加する事が出来ます。

In [31]:
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [32]:
# データ取り込み
train = pd.read_csv("titanic_train.csv")
test = pd.read_csv("titanic_test.csv")
data_check = pd.read_csv("titanic_gender_submission.csv")



# 列削除
train_x = train.drop(["Survived", "Name", "Cabin", "Ticket", "PassengerId"], axis=1)
test_x = test.drop(["Name", "Cabin", "Ticket", "PassengerId"], axis=1)


# Survived列のみ抽出
train_y = train["Survived"]


# 欠損値を年齢の中央値。出港地は最多のSで補完
train_x["Age"] = train_x["Age"].fillna(train_x["Age"].median())
train_x["Embarked"] = train_x["Embarked"].fillna("S")

test_x["Age"] = test_x["Age"].fillna(test_x["Age"].median())
test_x["Fare"] = test_x["Fare"].fillna(test_x["Fare"].median())
test_x["Embarked"] = test_x["Embarked"].fillna("S")


# Sex、Embarked列をダミー変換
train_x_dummy = pd.get_dummies(train_x,columns=["Sex","Embarked"])
test_x_dummy = pd.get_dummies(test_x,columns=["Sex","Embarked"])


# 説明変数、目的変数分割
X_train, X_test, y_train, y_test = train_test_split(train_x_dummy, train_y, test_size=0.3, random_state=101)

# Kerasを用いてtitanic生存予測<br>

①<br>
② ①　＋　Batch Normalization：バッチ正規化<br>
③ ②　＋　Dropout<br>
④ ③　＋　L2正則化<br>

In [33]:
# ①バッチサイズ、イテレーション数、エポック数


import keras
# from keras.modelsというのはモデルが色々はいったモジュール。その中から「Sequentialを使う」というのがimport Sequential
from keras.models import Sequential
# レイヤーはkeras.layersにまとめられていて、その中のDenseを使う事を上で宣言
from keras.layers import Dense, Dropout


model = Sequential()


# Sequentialが持っているaddメソッドでレイヤーを追加
# addメソッドの引数にDenseレイヤーを与える
# Denseレイヤーも多様なパラメータで初期化できるのですが、最も単純な初期化が上のようにunitsに数値を与えるだけのもの
# unitsはDenseを作る時に唯一必須のパラメータ
# Denseクラスは「ニューロンは次の層にあるあらゆるニューロンに全部連結する」という全連結層を作る時に使う
#　Denseは全結合するので、上のSequentialモデルの各レイヤーのユニットは次のユニットにすべて繋がる
# 入力シェイプ（input_shape）　N次元のデータ
model.add(Dense(units = 16, input_shape = (X_train.shape[1], )))
model.add(Dense(units = 24, activation='relu'))
model.add(Dense(16, activation='relu'))

model.add(Dense(units = 1, activation='sigmoid'))


# 平均二乗誤差（Mean squared error）, 重み更新方法の変更　：　確率的勾配降下法（Stochastic gradient descent）
model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics=['accuracy'])

model.fit( X_train, y_train, epochs = 500, batch_size = 30 , verbose=0)


predictions = model.predict(X_test, batch_size = 5)
np.set_printoptions(suppress=True)
predictions

# 2値分類のうちどちらに分類されるか
y_pred_proba = model.predict(X_test)
y_pred = np.where(y_pred_proba >0.5, 1, 0)

In [34]:
# 真陽性（TP: True Positive）: 実際のクラスが陽性で予測も陽性（正解）
# 真陰性（TN: True Negative）: 実際のクラスが陰性で予測も陰性（正解）
# 偽陽性（FP: False Positive）: 実際のクラスは陰性で予測が陽性（不正解）
# 偽陰性（FN: False Negative）: 実際のクラスは陽性で予測が陰性（不正解）

# クラス分類問題における評価指標
# 正解率（accuracy）: accuracy_score()
# 適合率（precision）: precision_score()
# 再現率（recall）: recall_score()
# F1値（F1-measure）: f1_score()

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

print('confusion matrix = \n', confusion_matrix(y_test, y_pred))
print('accuracy = ', accuracy_score(y_test, y_pred))
print('precision = ', precision_score(y_test, y_pred))
print('recall = ', recall_score(y_test, y_pred))
print('f1 score = ', f1_score(y_test, y_pred))

confusion matrix = 
 [[124  30]
 [ 33  81]]
accuracy =  0.7649253731343284
precision =  0.7297297297297297
recall =  0.7105263157894737
f1 score =  0.7200000000000001


In [35]:
# ②Batch Normalization：バッチ正規化
#model.add(BatchNormalization())

from keras.layers import Dense, BatchNormalization, Activation

model.add(Dense(units = 16, input_shape = (X_train.shape[1], )))
model.add(Dense(units = 24, activation='relu'))
model.add(Dense(16, activation='relu'))

# Batch Normalization：バッチ正規化
model.add(BatchNormalization())


model.add(Dense(units = 1, activation='sigmoid'))


# 平均二乗誤差（Mean squared error）, 重み更新方法の変更　：　確率的勾配降下法（Stochastic gradient descent）
# sgd = optimizers.SGD( learning_rate=0.001 ) 
model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics=['accuracy'])

model.fit( X_train, y_train, epochs = 500, batch_size = 30 , verbose=0)


predictions = model.predict(X_test, batch_size = 5)
np.set_printoptions(suppress=True)
predictions

# 2値分類のうちどちらに分類されるか
y_pred_proba = model.predict(X_test)
y_pred = np.where(y_pred_proba >0.5, 1, 0)

In [36]:
print('confusion matrix = \n', confusion_matrix(y_test, y_pred))
print('accuracy = ', accuracy_score(y_test, y_pred))
print('precision = ', precision_score(y_test, y_pred))
print('recall = ', recall_score(y_test, y_pred))
print('f1 score = ', f1_score(y_test, y_pred))

confusion matrix = 
 [[130  24]
 [ 40  74]]
accuracy =  0.7611940298507462
precision =  0.7551020408163265
recall =  0.6491228070175439
f1 score =  0.6981132075471698


In [37]:
# ③Dropout

from keras.layers import Dense, BatchNormalization, Activation

model.add(Dense(units = 16, input_shape = (X_train.shape[1], )))
model.add(Dense(units = 24, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(16, activation='relu'))

# Batch Normalization：バッチ正規化
model.add(BatchNormalization())


model.add(Dense(units = 1, activation='sigmoid'))


# 平均二乗誤差（Mean squared error）, 重み更新方法の変更　：　確率的勾配降下法（Stochastic gradient descent）
# sgd = optimizers.SGD( learning_rate=0.001 ) 
model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics=['accuracy'])

model.fit( X_train, y_train, epochs = 500, batch_size = 30 , verbose=0)


predictions = model.predict(X_test, batch_size = 5)
np.set_printoptions(suppress=True)
predictions

# 2値分類のうちどちらに分類されるか
y_pred_proba = model.predict(X_test)
y_pred = np.where(y_pred_proba >0.5, 1, 0)

In [38]:
print('confusion matrix = \n', confusion_matrix(y_test, y_pred))
print('accuracy = ', accuracy_score(y_test, y_pred))
print('precision = ', precision_score(y_test, y_pred))
print('recall = ', recall_score(y_test, y_pred))
print('f1 score = ', f1_score(y_test, y_pred))

confusion matrix = 
 [[143  11]
 [ 50  64]]
accuracy =  0.7723880597014925
precision =  0.8533333333333334
recall =  0.5614035087719298
f1 score =  0.6772486772486772


In [39]:
# ④正則化

from keras import regularizers

model.add(Dense(units = 16, kernel_regularizer=regularizers.l2(0.001), input_shape = (X_train.shape[1], )))
model.add(Dense(units = 24, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(16, activation='relu'))

# Batch Normalization：バッチ正規化
model.add(BatchNormalization())


model.add(Dense(units = 1, activation='sigmoid'))


# 平均二乗誤差（Mean squared error）, 重み更新方法の変更　：　確率的勾配降下法（Stochastic gradient descent）
# sgd = optimizers.SGD( learning_rate=0.001 ) 
model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics=['accuracy'])

model.fit( X_train, y_train, epochs = 500, batch_size = 30 , verbose=0)


predictions = model.predict(X_test, batch_size = 5)
np.set_printoptions(suppress=True)
predictions

# 2値分類のうちどちらに分類されるか
y_pred_proba = model.predict(X_test)
y_pred = np.where(y_pred_proba >0.5, 1, 0)

In [40]:
print('confusion matrix = \n', confusion_matrix(y_test, y_pred))
print('accuracy = ', accuracy_score(y_test, y_pred))
print('precision = ', precision_score(y_test, y_pred))
print('recall = ', recall_score(y_test, y_pred))
print('f1 score = ', f1_score(y_test, y_pred))

confusion matrix = 
 [[140  14]
 [ 45  69]]
accuracy =  0.7798507462686567
precision =  0.8313253012048193
recall =  0.6052631578947368
f1 score =  0.700507614213198


①
confusion matrix = <br>
 [[124  30]<br>
 [ 33  81]]<br>
accuracy =  0.7649253731343284<br>
precision =  0.7297297297297297<br>
recall =  0.7105263157894737<br>
f1 score =  0.7200000000000001<br>

②<br>
confusion matrix = <br>
 [[130  24]<br>
 [ 40  74]]<br>
accuracy =  0.7611940298507462<br>
precision =  0.7551020408163265<br>
recall =  0.6491228070175439<br>
f1 score =  0.6981132075471698<br>

③<br>
confusion matrix = <br>
 [[143  11]<br>
 [ 50  64]]<br>
accuracy =  0.7723880597014925<br>
precision =  0.8533333333333334<br>
recall =  0.5614035087719298<br>
f1 score =  0.6772486772486772<br>

④<br>
confusion matrix = <br>
 [[140  14]<br>
 [ 45  69]]<br>
accuracy =  0.7798507462686567<br>
precision =  0.8313253012048193<br>
recall =  0.6052631578947368<br>
f1 score =  0.700507614213198<br>

## ----------------------------------------------------------------------

In [41]:
#モデル作成
model = Sequential()
model.add( InputLayer( input_shape = ( 1, ) ) )
model.add( Dense( 1, activation='relu' ) )
sgd = optimizers.SGD( learning_rate=0.001 ) # learning_rate(学習率)の設定は大切！
model.compile( loss = 'mse', optimizer = sgd, metrics=["accuracy"] )

#概略表示
model.summary()

#訓練開始
train_x0 = numpy.array( [ 10, 14, 8, 16 ] )
train_x1 = numpy.array( [ 5, 7, 4, 8 ] )
history = model.fit( train_x0, train_x1, epochs = 30, verbose=0) #epochs(繰り返し数)大切

#実用テスト
test_x0 = numpy.array( [ 200, 1000 ] )
res = model.predict( test_x0 )
print( res )

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________
[[ 99.88799]
 [499.40933]]


In [42]:
print('confusion matrix = \n', confusion_matrix(y_test, y_pred))
print('accuracy = ', accuracy_score(y_test, y_pred))
print('precision = ', precision_score(y_test, y_pred))
print('recall = ', recall_score(y_test, y_pred))
print('f1 score = ', f1_score(y_test, y_pred))

confusion matrix = 
 [[140  14]
 [ 45  69]]
accuracy =  0.7798507462686567
precision =  0.8313253012048193
recall =  0.6052631578947368
f1 score =  0.700507614213198


In [43]:
# 正則化　kernel_regularizer=keras.regularizers.l1

#モデル作成
model = Sequential()
model.add( keras.layers.Dense( units=100, activation="relu", kernel_regularizer=keras.regularizers.l1(0.01), input_shape=(1,) ) )

model.add( Dense( 1, activation='relu' ) )
sgd = optimizers.SGD( learning_rate=0.001 ) # learning_rate(学習率)の設定は大切！
model.compile( loss = 'mse', optimizer = sgd, metrics=["accuracy"] )

#概略表示
model.summary()

#訓練開始
train_x0 = numpy.array( [ 10, 14, 8, 16 ] )
train_x1 = numpy.array( [ 5, 7, 4, 8 ] )
history = model.fit( train_x0, train_x1, epochs = 30, verbose=0) #epochs(繰り返し数)大切

#実用テスト
test_x0 = numpy.array( [ 200, 1000 ] )
res = model.predict( test_x0 )
print( res )

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 100)               200       
_________________________________________________________________
dense_56 (Dense)             (None, 1)                 101       
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________
[[0.]
 [0.]]
